#### Instructor Input Towards Endterm Review (Mail)
    -- The team can try sector dataset, 
    -- -- -- ledgar dataset from https://www.csie.ntu.edu.tw/~cjlin/libsvmtools/datasets/ website. 
    -- The team can try mnist type of dataset (in vector form). 
    -- Try proper Grid search based K fold c
    -- As downstream tasks of the sparse metric matrix learned, the team can check the usefulness of the learned metric matrix in clustering
    (check the paper: https://ai.stanford.edu/~ang/papers/nips02-metric.pdf)
    -- and KNN for classification (where the learned metric matrix can be used to compute distances)

#### Instructor's Comments: 
        outline -- good (The participant was reading content from script) 
        motivation: -- ok 
        S3ML: -- highlighted important aspects 
        related work: -- LMLL -- very very brief description given 
        -- not given in detail L1 penalized log determinant regularization 
        -- very very brief description given -- no ideas given S3ML 
        -- again motivated solution approach: neighborhood matrix: -- affinity matrix -- ok explanation 
        alternating minimization method -- linearization, duplication and derivation of algorithm not very clear 
        Implementation: wine, iris Eu and S3ML were implemented by the team 
        Some components of LMNN was implemented already -- so the team took it 
        code demo: good 
        (significant effort has gone into coding) 

        For end term review, the team has to try the following: 
        -- Bayesian optimization -- can be attempted but only if the libraries can be directly incorporated 
        -- The team has to understand the algorithm completely: 
        especially the derivations leading to the optimization problem and the derivation of ALM 
        -- The team should be performing experiments on datasets with large number of classes 
        -- Check algorithm scalability of classes. In this setting, try proper Grid search based K fold cv 
        –- The instructor will get back with some more details of experiments if it can be attempted 


#### TAs Comments:


##### Bheeshm's Comments:
        Related work is explained very quickly. Screenshots from the paper should not be used in the presentation slides.                
        Understanding is good but explanation should be improved. Some theoretical explanations were missed. 
        Code implementation is done. 
        Overall seems okay. 
        For final review try it on other datasets mentioned in the paper.
##### Rahul's Comments: 
        Outline explained; In-hand script was being used while presentation (a clear and straight readiing of the script is not appreciated the way it was being used in the initial part of presentation);
        Related works explained; 
        Screenshots from paper were used; 
        Confused in ALM algorithm (understanding at this level needs to be improved). 
        Codes for S3ML AND EU WERE implemented by him completely; 
        Queries regradiing the h_\sigma function were not handled correctly. 
        A proposal for applying bayesian optimization was made. 
        The understanding of optimization problem and algorithm need to be improved.



In [92]:
import numpy as np 
from sklearn.datasets import load_iris
from sklearn.datasets import load_wine

seed_value = 42  
wine = load_wine()       

print('Wine dataset shape:', wine.data.shape)
print("wine target shape:", wine.target.shape)
print('Unique classes: ',np.unique(wine.target))  
print("Instances per class:", np.bincount(wine.target))
print("Labeled instances per class after sampling:", (np.bincount(wine.target))//20 +1 )

Wine dataset shape: (178, 13)
wine target shape: (178,)
Unique classes:  [0 1 2]
Instances per class: [59 71 48]
Labeled instances per class after sampling: [3 4 3]


In [93]:
from sklearn.preprocessing import StandardScaler
import warnings

X = wine.data
y = wine.target
#  scaler = StandardScaler()
#  X = scaler.fit_transform(X)

print("X:  ", X)
print("y: " , y)

warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)


X:   [[1.423000e+01 1.710000e+00 2.430000e+00 1.560000e+01 1.270000e+02
  2.800000e+00 3.060000e+00 2.800000e-01 2.290000e+00 5.640000e+00
  1.040000e+00 3.920000e+00 1.065000e+03]
 [1.320000e+01 1.780000e+00 2.140000e+00 1.120000e+01 1.000000e+02
  2.650000e+00 2.760000e+00 2.600000e-01 1.280000e+00 4.380000e+00
  1.050000e+00 3.400000e+00 1.050000e+03]
 [1.316000e+01 2.360000e+00 2.670000e+00 1.860000e+01 1.010000e+02
  2.800000e+00 3.240000e+00 3.000000e-01 2.810000e+00 5.680000e+00
  1.030000e+00 3.170000e+00 1.185000e+03]
 [1.437000e+01 1.950000e+00 2.500000e+00 1.680000e+01 1.130000e+02
  3.850000e+00 3.490000e+00 2.400000e-01 2.180000e+00 7.800000e+00
  8.600000e-01 3.450000e+00 1.480000e+03]
 [1.324000e+01 2.590000e+00 2.870000e+00 2.100000e+01 1.180000e+02
  2.800000e+00 2.690000e+00 3.900000e-01 1.820000e+00 4.320000e+00
  1.040000e+00 2.930000e+00 7.350000e+02]
 [1.420000e+01 1.760000e+00 2.450000e+00 1.520000e+01 1.120000e+02
  3.270000e+00 3.390000e+00 3.400000e-01 1.97000

In [94]:
import numpy as np
def select_5_percent_samples(X, y, seed):
    np.random.seed(seed)  # Seed for reproducibility

    # Determine the number of samples to select from each class (5%)
    num_samples_per_class = {label: int(np.ceil(0.05 * np.sum(y == label))) for label in np.unique(y)}

    # Initialize arrays to store the indices of labeled and unlabeled data
    labeled_indices = np.array([], dtype=int)
    unlabeled_indices = np.array([], dtype=int)

    # Select 5% from each class as labeled data
    for label in np.unique(y):
        indices = np.where(y == label)[0]  # Get indices of all samples belonging to 'label'
        np.random.shuffle(indices)
        num_labeled = num_samples_per_class[label]
        labeled_indices = np.append(labeled_indices, indices[:num_labeled])
        unlabeled_indices = np.append(unlabeled_indices, indices[num_labeled:])

    # Return labeled indices and their labels, along with unlabeled indices
    return labeled_indices, y[labeled_indices], unlabeled_indices , y[unlabeled_indices]

print(select_5_percent_samples(X, y, seed_value))

(array([  0,   5,  34, 116, 125,  91,  78, 158, 172, 137]), array([0, 0, 0, 1, 1, 1, 1, 2, 2, 2]), array([ 13,  45,  53,  57,  25,  47,  12,  49,   3,  36,  31,   8,  17,
         6,   4,  54,  19,  30,  50,  44,  55,  15,   9,  27,  26,  16,
        24,  33,  56,  41,  11,  32,  52,  40,  37,  29,  43,   1,  21,
         2,  46,  39,  35,  23,  48,  10,  22,  18,  58,  20,   7,  42,
        14,  28,  51,  38, 101, 119,  71,  96, 120,  80, 103,  99,  86,
       106, 111,  90, 109,  63, 123, 117,  61,  85,  75,  77,  69,  88,
       104,  70, 110,  97,  67,  65,  95,  59, 126, 114,  83, 115,  82,
       113,  74,  79, 122,  81, 107, 127,  73,  89,  72, 124,  68,  92,
       128,  84,  76,  87, 118, 121, 100, 112,  64,  60,  62,  98, 108,
        94,  93, 105,  66, 102, 129, 141, 151, 150, 171, 145, 147, 157,
       135, 163, 174, 168, 161, 139, 131, 153, 154, 167, 140, 175, 162,
       159, 152, 170, 133, 149, 146, 136, 160, 148, 142, 169, 165, 144,
       138, 156, 173, 155, 134, 176, 

In [95]:
from sklearn.neighbors import NearestNeighbors
k = 6 ## As sugested in the paper, page 7

def construct_neighborhood_indicator_matrix(X):

    # Initialize the nearest neighbors model and fit it to the labeled data
    nn = NearestNeighbors(algorithm='auto', metric="euclidean", n_neighbors=6).fit(X)

    distances, indices = nn.kneighbors(X) ## For this matrix, we are not interested in distances, only indices.

    # Initialize the neighborhood indicator matrix P with zeros
    P = np.zeros((len(X), len(X)))

    # Fill in the neighborhood indicator matrix P for the labeled samples
    for i in range(len(X)):
        P[i, indices[i]] = 1 / k
    
    return P


In [96]:
# Select 5% of the samples from each class as labeled
X_labeled, y_labeled, X_unlabeled, y_unlabeled = select_5_percent_samples(X, y, seed_value)

# Construct neighborhood indicator matrix P for the labeled data
P = construct_neighborhood_indicator_matrix(X)

print("P: ", P)  # Show the constructed matrix P for verification
row_sums = np.sum(P, axis=1)
print("Row sums: ", row_sums)  # Verify that the row sums are equal to 1

P:  [[0.16666667 0.16666667 0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.16666667 0.16666667 0.
  0.16666667 0.         0.         0.         0.         0.
  0.16666667 0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0

In [97]:
alpha = 0.5  ## Damping factor
teta = 0.01  ## Treshold value to determine the strong affinities.
## These two paramerer values are also mentioned in the paper, p.7 

def affinity_propagation(P,labeled_indices, y_labeled, alpha, teta):
    # Initialize the affinity matrix W_0

    n_samples = P.shape[0]
    W_0 = np.zeros((n_samples, n_samples)) ## np.eye(n_samples) ## da olabilir. Tekrar iyi bak.
    
    ## Initially creates a matix full of zeros with the respective dimensions
    # Set affinities for similar and dissimilar pairs
    for i, idx_i in enumerate(labeled_indices):
        for j, idx_j in enumerate(labeled_indices):
            if y_labeled[i] == y_labeled[j]: ## Initially we mark similar points as if in 
                W_0[idx_i, idx_j] = 1  ## Similar pairs, as written in the paper
            else:
                W_0[idx_i, idx_j] = -1  ## Dissimilar pairs
    
    ## W0 has been created using %5 labeled indices. 
    ## Now we need to propagate the affinities through the neighborhood structure.
    # Propagate affinities through the neighborhood structure.
    W = np.zeros_like(W_0) ##Creates an empty matrix with the same dimensions as W_0.

    for _ in range(n_samples):
        W = (1 - alpha) * W_0 + alpha * np.dot(P, W)
        
        # Apply the threshold to determine strong affinities
        W[np.abs(W) < teta] = 0
    
    return W

In [98]:
def step_2(W,X):

    W1 = np.sum(W, axis=1)  # This computes the sum of each row (axis=1 for rows)
    D = np.diag(W1)  # This creates a diagonal matrix D from the vector W1

    # Compute the graph Laplacian L
    L = D - W

    # Compute the matrix T
    T = X.T @ L @ X  ##  @ stands for matrix multiplication in numpy

    return T

## Now set, Σ = np.linalg.inv(M0) + beta * T  

In [99]:
import numpy as np
# Use the function to perform affinity propagation

alpha = 0.5  # Example value for the damping factor
threshold = 0.01  # Example threshold value for strong affinities

W = affinity_propagation(P, X_labeled, y_labeled, alpha, teta)
T = step_2(W,X) ## It is basically an (n * n) square matrix where n = #features of X

## Verify the affinity matrix

np.set_printoptions(threshold=np.inf)

print(W)

print("W: ",W)  
#print("T: ", T)
print("W_T: ", W.transpose)
#print("P shape: ", P.shape)
print("W shape: ", W.shape)
#print("T shape: ", T.shape)


[[ 0.55635501  0.          0.          0.          0.          0.55635501
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.55635501  0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
  -0.55635501  0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
  

In [100]:
def h_sigma(M, rho, sigma): # Apply Nesterov's smoothing technique to smooth the l1 term
    U_star = np.minimum(rho, np.maximum(M / sigma, -rho))
    return np.trace(U_star.T @ M) - (sigma / 2) * np.linalg.norm(U_star, 'fro')**2

def grad_h_sigma(M, rho, sigma):
    return np.minimum(rho, np.maximum(M / sigma, -rho))

def f(M, Sigma):
    fM = -np.log(np.linalg.det(M)) + np.trace(Sigma @ M)
    return fM

def grad_f(M,Σ):
    grad_fM = -np.linalg.inv(M) + Σ
    return grad_fM

def Σ(X,y_labeled, alpha, beta):
    M0 = np.eye(X.shape[1])  # Initialize M0 as the identity matrix
    P = construct_neighborhood_indicator_matrix(X)
    W = affinity_propagation(P, y_labeled, alpha, teta)
    T = step_2(W)
    Σ = np.linalg.inv(M0) + beta * T  
    return Σ

In [101]:
def ALM(M_init, Y_init, mu, rho, sigma, Σ, max_iter=500):
    M, Y = M_init, Y_init
    
    for _ in range(max_iter):
        grad_h_Y = grad_h_sigma(Y, rho, sigma) # Step 1: Update M
        M_next = M - mu * (grad_h_Y + grad_f(M,Σ))
        
        grad_f_M_next = grad_f(M_next,Σ) # Step 2: Update Y
        Y_next = Y - mu * (grad_f_M_next + grad_h_sigma(M_next, rho, sigma))   
        
        if np.linalg.norm(M_next - M, 'fro') < 1e-6:
            break
        
        M,Y = M_next,Y_next # Prepare for next iteration
    return M

In [104]:
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier

rho = 3 ## Tuning (smoothness parameter)
beta = 3.5    ## Tuning parameter

mu = 0.000001  # Update step size
sigma = 0.000001   # Smoothness parameter

############################ check down below for debugging / compatibilitiy issues

def run_s3ml(X_labeled, y_labeled, X_unlabeled, y_unlabeled, M0, Y0, mu, rho, sigma, beta):
    # Construct neighborhood indicator matrix P for the labeled data
    P = construct_neighborhood_indicator_matrix(X)
    # Apply affinity propagation to obtain the affinity matrix W
    W = affinity_propagation(P, X_labeled, y_labeled, alpha, teta)
    # Compute matrix T
    T = step_2(W,X)
    # Set Σ = np.linalg.inv(M0) + beta * T  
    Σ = np.linalg.inv(M0) + beta * T

    # Run ALM to obtain the sparse metric matrix M
    M = ALM( M0, Y0, mu, rho, sigma, Σ)

    # Fit kNN using the labeled data
    knn = KNeighborsClassifier(n_neighbors=1, metric="mahalanobis", metric_params={'VI': np.linalg.inv(M)})
    knn.fit(X[X_labeled], y_labeled)  # Fit the model with labeled data

    # Now predict on the unlabeled data
    # Get predictions for the unlabeled data
    predictions = knn.predict(X[X_unlabeled])

    # Calculate accuracy score and error rate
    accuracy = accuracy_score(y_unlabeled, predictions)

    error_rate = 1 - accuracy
    print("S3ML Error rate: ", error_rate)
    return error_rate


def experiment_S3ML(experiment_number,  seed_value,mu, rho, sigma, beta, avg_S3ML_error_rate):
    for _ in range(experiment_number):
        X_labeled, y_labeled, X_unlabeled , y_unlabeled = select_5_percent_samples(X, y, seed_value)
        M0 = np.identity(X.shape[1])
        Y0 = np.zeros_like(M0)

        avg_S3ML_error_rate += run_s3ml(X_labeled, y_labeled, X_unlabeled,y_unlabeled, M0, Y0, mu, rho, sigma, beta)
        seed_value += 1
        
    avg_S3ML_error_rate = avg_S3ML_error_rate/experiment_number

    print("Average error rate of S3ML after ", experiment_number ,"tests: ", avg_S3ML_error_rate)

experiment_S3ML(50, seed_value, mu, rho, sigma, beta,0)

## Experiments done on random hyperparameters, the results do not reflect the true performance of the algorithm.

S3ML Error rate:  0.8273809523809523
S3ML Error rate:  0.45833333333333337
S3ML Error rate:  0.24404761904761907
S3ML Error rate:  0.8154761904761905
S3ML Error rate:  0.6845238095238095
S3ML Error rate:  0.38690476190476186
S3ML Error rate:  0.4821428571428571
S3ML Error rate:  0.6666666666666667
S3ML Error rate:  0.875
S3ML Error rate:  0.6547619047619048
S3ML Error rate:  0.45238095238095233
S3ML Error rate:  0.36309523809523814
S3ML Error rate:  0.25
S3ML Error rate:  0.8333333333333334
S3ML Error rate:  0.8928571428571429
S3ML Error rate:  0.5357142857142857
S3ML Error rate:  0.7619047619047619
S3ML Error rate:  0.2321428571428571
S3ML Error rate:  0.5535714285714286
S3ML Error rate:  0.8630952380952381
S3ML Error rate:  0.5535714285714286
S3ML Error rate:  0.8928571428571429
S3ML Error rate:  0.9345238095238095
S3ML Error rate:  0.8095238095238095
S3ML Error rate:  0.29761904761904767
S3ML Error rate:  0.4464285714285714
S3ML Error rate:  0.23809523809523814
S3ML Error rate:  0.4

In [105]:
###     Grid search (extra):
def search_optimal_parameters(X, y, seed_value, mu, sigma, start_value=3.5, rho_max_value=6, beta_max_value=4.2, experiment_number=9):
    best_rho = start_value
    best_beta = start_value
    best_error_rate = float('inf')

    rho = start_value
    beta = start_value

    while rho <= rho_max_value or beta <= beta_max_value:
        avg_error_rate = 0
        for _ in range(experiment_number):
            X_labeled, y_labeled, X_unlabeled, y_unlabeled = select_5_percent_samples(X, y, seed_value)
            M0 = np.identity(X.shape[1])
            Y0 = np.zeros_like(M0)

            error_rate = run_s3ml(X_labeled, y_labeled, X_unlabeled, y_unlabeled, M0, Y0, mu, rho, sigma, beta)
            seed_value += 1
            avg_error_rate += error_rate

        avg_error_rate /= experiment_number

        print(f"rho: {rho}, beta: {beta}, avg error rate: {avg_error_rate}")

        if avg_error_rate < best_error_rate:
            best_error_rate = avg_error_rate
            best_rho = rho
            best_beta = beta

        if rho <= rho_max_value:
            rho *= 1.03
        elif beta <= beta_max_value:
            beta *= 1.01
            rho = 1  # Reset rho to 1 when beta increases

        if rho > rho_max_value and beta > beta_max_value:
            break

    return best_rho, best_beta, best_error_rate

# Initial parameters
mu = 1e-6
sigma = 1e-6
seed_value = 42

best_rho, best_beta, best_error_rate = search_optimal_parameters(X, y, seed_value, mu, sigma)
print(f"Best rho: {best_rho}, Best beta: {best_beta}, Best error rate: {best_error_rate}")



S3ML Error rate:  0.5416666666666667
S3ML Error rate:  0.29761904761904767
S3ML Error rate:  0.1607142857142857
S3ML Error rate:  0.19047619047619047


KeyboardInterrupt: 

In [ ]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

def random_search_s3ml(X, y, folds, num_samples, rho_range, beta_range, mu, sigma, seed_value):
    np.random.seed(seed_value)
    kf = KFold(n_splits=folds, shuffle=True, random_state=seed_value)
    best_rho = None
    best_beta = None
    best_error_rate = float('inf')

    for _ in range(num_samples):
        rho = np.random.uniform(*rho_range)
        beta = np.random.uniform(*beta_range)
        error_rates = []

        for train_index, test_index in kf.split(X):
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]

            # Select samples and run the model
            X_labeled, y_labeled, X_unlabeled, y_unlabeled = select_5_percent_samples(X_train, y_train, seed_value)
            M0 = np.identity(X.shape[1])
            Y0 = np.zeros_like(M0)

            error_rate = run_s3ml(X_labeled, y_labeled, X_unlabeled, y_unlabeled, M0, Y0, mu, rho, sigma, beta)
            error_rates.append(error_rate)
            seed_value += 1

        avg_error_rate = np.mean(error_rates)
        if avg_error_rate < best_error_rate:
            best_error_rate = avg_error_rate
            best_rho = rho
            best_beta = beta

        print(f"Sampling: rho={rho:.2f}, beta={beta:.2f}, Avg Error Rate={avg_error_rate:.4f}")

    return best_rho, best_beta, best_error_rate

# Example usage
rho_range = (1e-2, 1e4)  # Define range for rho
beta_range = (1e-2, 1e2)  # Define range for beta
mu = 1e-6
sigma = 1e-6
seed_value = 42
folds = 10
num_samples = 100  # Number of random samples to test

best_rho, best_beta, best_error_rate = random_search_s3ml(X, y, folds, num_samples, rho_range, beta_range, mu, sigma, seed_value)
print(f"Best rho: {best_rho}, Best beta: {best_beta}, Best error rate: {best_error_rate:.4f}")

In [ ]:
## Buradan bir hocanın dediğini yapmaya çalışsana....
